# Inference of hierarchical stochastic block models (hSBMs) on RS-fMRI data

This Python-notebook is the second step in the analysis of RS-fMRI data with hierarchical stochastic block models. It is part of the analyses underlying the dissertation "Topic modelling for the stratification of neurological patients" written by W. Van Echelpoel (WVE) under supervision of prof. D. Marinazzo (DM) (Ghent University). The data has been provided by DM and has been pre-processed through another Python-notebook ('S02_DataSelection.ipynb'). The reason for using two different notebooks is related to the used OS: Data pre-processing could be done locally (Anaconda-distribution in Windows OS), yet inference of hSBM requires a specific module for which native installation on Windows is not supported. Hence, this notebook was developed to work in the Google Colab environment, relying on a Linux OS. It is advised to take this into account when willing to use this notebook to repeat our analyses or for performing other analyses.

The notebook has been developed to work with the pre-processed data obtained through the notebook 'S02_DataSelection.ipynb' and stored in a specific folder structure. It is assumed that this folder structure is copied to GoogleDrive prior to attempting to read in this data. Alternative structures are possible, but require changes in the notebook. 

Visualisations of the obtained networks and cluster membership distributions are provided to get an insight, yet final graphs for cluster membership have been developed in R ('S05_GraphsMembership.R').

This notebook is in many aspects based on:
- The documentation of graphtool: https://graph-tool.skewed.de/static/doc/index.html
- The tutorial of Valle et al. (2016): https://github.com/fvalle1/hSBM_Topicmodel
- The study of Valle et al. (2020): https://github.com/fvalle1/topicTCGA


## Preparations

In [ ]:
# First, create environment with required modules
!echo "deb http://downloads.skewed.de/apt focal main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib libcairo2-dev python3-cairo libgtk-3-dev

# Colab uses a Python install that deviates from the system's! We need some workarounds.
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

In [ ]:
# Preparation of analyses
%load_ext autoreload
%autoreload 2

import os
import sys
import math
import random
import numpy as np
import pandas as pd
import pylab as plt
import graph_tool.all as gt
from google.colab import drive
from collections import defaultdict
from sklearn import metrics
%matplotlib inline 

In [ ]:
# Connect with GoogleDrive
drive.mount('/content/gdrive')

## Loading the data

In [ ]:
# Prepare loading data from Drive (complete ('NoNa') versus all data)
path_data = '/content/gdrive/My Drive/Analysis/Data/02 Cleaned data'
fname_data = ['D_PearsonCoefficient_NoNa.txt', 'D_PearsonCoefficient.txt', 
              'D_SelectedPairs_NoNa_75.txt', 'D_SelectedPairs_75.txt', 
              'D_SelectedPairs_NoNa_50.txt', 'D_SelectedPairs_50.txt'][5]
filename = os.path.join(path_data,fname_data)

# Read in data and extract info
corrDF = pd.read_csv(filename, sep = ";", index_col = 0)
lst_patients = corrDF.index.values.tolist()
lst_pairs = corrDF.columns.values.tolist()

# Define treshold (MANUALLY!) --> '1' is selected for dynamic threshold
n_lmtPearson = [0, 0.1, 0.25, 1][0]

In [ ]:
# Make use of dynamic threshold
if n_lmtPearson == 1:
    # Determine number of edges (1.5 times number of nodes) and derive index
    n_indx = round(3 * (len(lst_patients) + len(lst_pairs)) / 2.0) - 1
    n_lmtPearson = float(math.floor(1000 * sorted(abs(corrDF).stack().tolist(), reverse = True)[n_indx])) * 0.001
    print(n_lmtPearson)

In [ ]:
# Select subset if interested (MANUALLY!)
# corrDF = corrDF.iloc[:120,] # Healthy
# corrDF = corrDF.iloc[120:170,] # Schizophrenia
# corrDF = corrDF.iloc[170:219,] # Bipolar
# corrDF = corrDF.iloc[219:,] # ADHD

In [ ]:
corrDF.shape

# Building the graph - manual creation

In [ ]:
# Create a graph
g = gt.Graph(directed=False)
# Define node properties
# name: patients - id, pair - 'pair'
# kind: patients - 0, pair - 1
name = g.vp["name"] = g.new_vp("string")
kind = g.vp["kind"] = g.new_vp("int")
eweight = g.ep["weight"] = g.new_ep("double")

pats_add = defaultdict(lambda: g.add_vertex())
pair_add = defaultdict(lambda: g.add_vertex())

In [ ]:
# Add all patients first
for i_d in range(corrDF.shape[0]):
    patient = lst_patients[i_d]
    p = pats_add[patient]

In [ ]:
# Add all patients and pairs as nodes, Pearson as links
for i_d in range(corrDF.shape[0]):
    # Recall patient from vertex dictionary
    patient = lst_patients[i_d]
    p = pats_add[patient]
    name[p] = patient
    kind[p] = 0

    # Add (or recall) ROI-pairs and add Pearson as edge weight
    for j_d in range(len(lst_pairs)):
      r = pair_add[lst_pairs[j_d]]
      name[r] = lst_pairs[j_d]
      kind[r] = 1

      # Add weight to edge
      if abs(corrDF.iloc[i_d][j_d]) >= n_lmtPearson:
        e = g.add_edge(p, r)
        eweight[e] = corrDF.iloc[i_d][j_d]

## Fit model

In [ ]:
# Extract vertex properties (patient or ROI-pair), as constraint for next step
# Both for the clustering (clabel) as for partition description length (pclabel)
clabel = g.vp['kind']
state_args = {'clabel': clabel, 'pclabel': clabel}

In [ ]:
n_init = 1
v_mdl = []
n_mdl = np.inf
gt.seed_rng(32) ## seed for graph-tool's random number generator --> same results

for i_n_init in range(n_init):
    print(i_n_init + 1)
    base_type = gt.BlockState
    state_tmp = gt.minimize_nested_blockmodel_dl(g, state_args=dict(
        base_type=base_type, **state_args, 
        recs=[g.ep.weight], rec_types=["real-normal"]))
    L = 0
    for s in state_tmp.levels:
          L += 1
          if s.get_nonempty_B() == 2:
                break
    state_tmp = state_tmp.copy(bs=state_tmp.get_bs()[:L] + [np.zeros(1)])
    print(state_tmp)

    mdl_tmp = state_tmp.entropy()
    v_mdl.append(mdl_tmp)
    if mdl_tmp < n_mdl:
        n_mdl = 1.0*mdl_tmp
        state = state_tmp.copy()

In [ ]:
# Visualise inferred network structure
state.draw(layout = "bipartite", sample_edges = 1000)

In [ ]:
# Save graph if interested, name to be defined (MANUALLY!)
file_out = "F_Temporary.png"
path_output = '/content/gdrive/My Drive/Analysis/Data/03 Analysed data/NetworkGraphs'
state.draw(layout = "bipartite", sample_edges = 1000, 
           output = os.path.join(path_output, file_out))

### Characteristics

In [ ]:
# Extract characteristics (note reduction of MDL)
n_mdl*0.000001, L

In [ ]:
# Define level to be studied (MANUALLY!)
n_lvl = 0

In [ ]:
## Define group/cluster membership
# Extract number of patients, roi-pairs, and edges
D, V, N = int(np.sum(g.vp['kind'].a==0)), int(np.sum(g.vp['kind'].a==1)), int(g.num_edges())

# Extract state at specific level
state_l = state.project_level(l=n_lvl).copy(overlap=True)
b = gt.contiguous_map(state_l.b)

label_map = {}
for v in g.vertices():
    label_map[state_l.b[v]] = b[v]
state_l = state_l.copy(b=b)

state_l_edges = state_l.get_edge_blocks() ## labeled half-edges

weights = 'weight' in g.ep.keys()

B = state_l.get_nonempty_B()

n_wb = np.zeros((V, B))  ## number of half-edges incident on pair-node w and labeled as pair-group tw
n_db = np.zeros((D, B))  ## number of half-edges incident on patient-node d and labeled as patient-group td
n_dbw = np.zeros((D, B)) ## number of half-edges incident on patient-node d and labeled as pair-group td

eweight = g.ep["weight"]

ze = gt.ungroup_vector_property(state_l_edges, [0,1])
for v1, v2, z1, z2, w in g.get_edges([ze[0], ze[1], eweight]):
    n_db[int(v1), int(z1)] += w
    n_dbw[int(v1), int(z2)] += w
    n_wb[int(v2 - D), int(z2)] += w

ind_d = np.where(np.sum(n_db, axis=0) > 0)[0]
Bd = len(ind_d)
n_db = n_db[:, ind_d]

ind_w = np.where(np.sum(n_wb, axis=0) > 0)[0]
Bw = len(ind_w)
n_wb = n_wb[:, ind_w]

# Group memberships of each pair-node P(t_w | w) and patient node P(t_d | d)
p_tw_w = (n_wb / np.sum(n_wb, axis=1)[:, np.newaxis]).T
p_td_d = (n_db / np.sum(n_db, axis=1)[:, np.newaxis]).T

In [ ]:
# Extract number of clusters
len(p_td_d)

In [ ]:
# Determine Normalised Mutual Information (NMI) statistic
# Note that this is not possible when dealing with subsets
v_true = [1 for i in range(120)] + [2 for i in range(50)] + [3 for i in range(49)] + [4 for i in range(40)]
v_pred = np.matmul(p_td_d.transpose(), [i + 1 for i in range(len(p_td_d))])

n_nmi = metrics.normalized_mutual_info_score(v_true, v_pred) # Also in R possible, with 'variant = "sqrt"'
n_nmi

In [ ]:
# Compare NMI with null-statistic (random sampling)
# Note that this is not possible when dealing with subsets
v_nmi_null = []
for i_d in range(10):
  random.seed(i_d)
  v_null = v_pred[random.sample([i for i in range(len(v_pred))], len(v_pred))]
  v_nmi_null.append(metrics.normalized_mutual_info_score(v_true, v_null))

np.mean(v_nmi_null), np.std(v_nmi_null), n_nmi / np.mean(v_nmi_null)

In [ ]:
# Plot memberships in two separate figures
plt.figure(figsize=(15,4))

plt.subplot(121)
plt.imshow(p_td_d,origin='lower',aspect='auto',interpolation='none')
plt.title(r'Participant group membership $P(bp | p)$')
plt.xlabel('Participant p (index)')
plt.yticks(ticks=range(len(p_td_d)),labels=range(1,len(p_td_d)+1))
plt.ylabel('Participant group, bp')
plt.colorbar()

plt.subplot(122)
plt.imshow(p_tw_w,origin='lower',aspect='auto',interpolation='none')
plt.title(r'ROI-pair group membership $P(br | r)$')
plt.xlabel('ROI-pair r (index)')
plt.yticks(ticks=range(len(p_td_d)),labels=range(1,len(p_td_d)+1))
plt.ylabel('ROI-pair group, br')
plt.colorbar()

In [ ]:
# Save participant membership if interested, name to be defined (MANUALLY!)
file_out = 'D_Temporary.txt'
path_output = '/content/gdrive/My Drive/Analysis/Data/03 Analysed data/MembershipData'
np.savetxt(os.path.join(path_output, file_out), p_td_d)